In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [3]:
#pgd.search_name("tetrapod")
pgd.search_name("primates")

,name,id
0,Primates,15963


In [4]:
#pgd.fetch_tol_data(15963, limit=25000)

In [5]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [6]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [7]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [8]:
pgd.load_prep_data("data/Mammalia-15040-df.csv")

### return data for plotting

In [9]:
df = pgd.return_data()
print(df.shape)
df.head()

(1688, 11)


,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
0,14973,1,0,15040,Mammalia,2,0,0.985482,-0.609871,225.00,0.0
1,15040,2,2,15989,Triconodonts,0,0,0.626042,-1.029422,190.00,70.0
2,15040,2,0,15991,Monotremata,2,0,1.577066,-1.157830,201.30,201.3
3,15991,3,0,16250,Tachyglossidae,4,0,1.934838,-1.035496,23.03,0.0
4,16250,4,0,16251,Tachyglossus aculeatus,0,0,2.682316,-1.617251,NaN,NaN


### hack: get rid of rows with no Begin time or too big Begin time

In [10]:
#df.sort_values(by='Begin', ascending=False).head()

In [11]:
#df = df.loc[df['Begin'].notnull()]
#df = df.loc[df['Begin'] < 500]
#print(df.shape)

## Plot Data

In [12]:
pgp = ph.phyloGraph(df)

the plotting of phylo


In [13]:
pgp.search_name('rrhi')

              name     id
443    Platyrrhini  16292
460     Catarrhini  16293
525  Strepsirrhini  16295


In [14]:
#focus = 'all'
focus = 16293

In [15]:
pick = focus
kin = [pick]
# add parents
kin += pgp.links_dict[focus]['parents']
# add kids
for c in pgp.links_dict[focus]['children']:
    kin += pgp.links_dict[c]['children']
kin = list(set(kin))
#kin

In [16]:
#pgp.create_plot_data(max_depth=30, max_nodes=50000, highlight=focus, Z_dim='Begin', Z_dim_mult=1)
pgp.create_plot_data(max_depth=30, max_nodes=50000, highlight=focus, add_links=True)
#pgp.create_plot_data(max_depth=30, max_nodes=50000, highlight=focus)

/Users/Seth/Documents/tolGraph/phyloGraph.py:440: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



     index  ancestor  depth  extinct      id                      name  \
442    442     15997      3        0   15963                  Primates   
460    460     15963      4        0   16293                Catarrhini   
462    462     16297      6        0   16383                    Macaca   
463    463     16297      6        0   16386                Cercocebus   
464    464     16297      6        0   16387                Mandrillus   
465    465     16297      6        0   16389                     Papio   
466    466     16297      6        0   16390             Theropithecus   
467    467     16297      6        0   16394             Cercopithecus   
468    468     16297      6        0   16395              Erythrocebus   
469    469     16297      6        0   16396               Miopithecus   
470    470     16297      6        0   16397            Allenopithecus   
471    471     16297      6        0   16400                  Collobus   
472    472     16297      6        0  

In [18]:
self = pgp 
highlight = 'all'
max_nodes = 5000
max_depth = 20
color_attr = 'extinct'
Z_dim = 'depth'
Z_dim_mult = -1
add_links = False
''''''

''

In [19]:
if highlight == 'all':
    pick = self.df.ix[0, 'id']
else:
    pick = highlight

# subset to max_nodes and max_depth
df = self.df[self.df['depth'] <= max_depth].head(max_nodes)
df = df.reset_index()

# get kinfolk
kin = [pick]
# add parents
kin += self.links_dict[pick]['parents']
# add kids
for c in self.links_dict[pick]['children']:
    kin += self.links_dict[c]['children']
kin = list(set(kin))

In [20]:
# subset
df['kin'] = 0
df['kin'][df['id'].isin(kin)] = 1
print(df[df['kin']==1])

      index  ancestor  depth  extinct     id                      name  \
0         0     14973      1        0  15040                  Mammalia   
3         3     15991      3        0  16250            Tachyglossidae   
8         8     15991      3        0  16253  Ornithorhynchus anatinus   
11       11     15994      3        0  16242           Didelphimorphia   
12       12     15994      3        0  16243          Paucituberculata   
13       13     15994      3        0  16244            Microbiotheria   
14       14     15994      3        0  16245            Dasyuromorphia   
15       15     15994      3        0  16246           Peramelemorphia   
16       16     15994      3        0  16247          Notoryctemorphia   
17       17     15994      3        0  16248             Diprotodontia   
20       20     15997      3        0  15953                  Edentata   
27       27     15997      3        0  15954                 Pholidota   
44       44     15997      3        0 

/Users/Seth/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [ ]:
###############
### NEED TO CREATE LINKS_LIST HERE
## create links list
links_list = []
for i, n in df.iterrows():
    links_list.append({
        'source': n['id'], 
        'target': n['ancestor'], 
        'value': n['num_kids']
    })

#print("len(links_list): {}".format(len(links_list)))
#print("first 3 links: {}".format(links_list[:3]))

#links_list = [l for l in links_list if l['target'] in list(df['id'])]
L=len(links_list)
#print("len(links_list): {}".format(L))

#
Edges=[(links_list[k]['source'], links_list[k]['target']) for k in range(L)]
#print("Edges {}".format(Edges[:5]))

# 
labels=[]
group=[]
alpha = []
layt = []
for i, node in df.iterrows():
    # create text labels
    if add_links:
        try:
            #this_page_id = wikipedia.search(node['name'], results=1)[0]
            labels.append('<a href="https://en.wikipedia.org/wiki/{}">{}</a>'.format(node['name'], node['name']))
        except:
            labels.append(str(node['name']))
    else:
        labels.append(str(node['depth']) + ' ' + node['name'])
    # create color key
    group.append(node[color_attr])
    # create opacity key
    alpha.append(node['kin'])
    # create layout list
    d = node[Z_dim]
    # # PCA
    # # layt.append(list(X_pca[i]) + [d * -1])
    layt.append([node['x'], 
                 node['y'], 
                 (d*Z_dim_mult)+np.random.uniform(-0.1,0.1,1)[0]])    


#print(labels[:3])
#print(group[:3])
#print(alpha[:3])
#print(len(layt))
#print(layt[:3])

# make nodes
Xn=[layt[k][0] for k in range(len(layt))]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(len(layt))]# y-coordinates
Zn=[layt[k][2] for k in range(len(layt))]# z-coordinates

# check if they're kinfolk
Xnk=[layt[k][0] for k in range(len(layt)) if alpha[k]==1]# x-coordinates of nodes
Ynk=[layt[k][1] for k in range(len(layt)) if alpha[k]==1]# y-coordinates
Znk=[layt[k][2] for k in range(len(layt)) if alpha[k]==1]# z-coordinates
groupk=[group[k] for k in range(len(layt)) if alpha[k]==1]# color
#print(len(Xnk))

# make edges
Xe=[]
Ye=[]
Ze=[]
Xek=[]
Yek=[]
Zek=[]
for e in Edges:
    try: 
        e0 = df[df['id'] == e[0]].index.values[0]
    except:
        e0 = 0
    try:
        e1 = df[df['id'] == e[1]].index.values[0]
    except:
        e1 = 0
    #
    Xe+=[layt[e0][0],layt[e1][0], None]# x-coordinates of edge ends
    Ye+=[layt[e0][1],layt[e1][1], None]  
    Ze+=[layt[e0][2],layt[e1][2], None] 
    # check if they're kinfolk
    if e[0] in kin:
        Xek+=[layt[e0][0],layt[e1][0], None]
        Yek+=[layt[e0][1],layt[e1][1], None]
        Zek+=[layt[e0][2],layt[e1][2], None]

In [17]:
#pgp.plot_data
pgp.render_plot()
#pgp.render_plot(publish=True, filename="primates2")

In [13]:
#pgp.open_plot()